In [1]:
import oracledb

connection = oracledb.connect(
    user = 'BD150224213',
    password = 'Hhqnm9',
    dsn = '172.16.12.14/xe',
)


In [2]:
cursor = connection.cursor()


In [9]:
cursor = connection.cursor()

cursor.execute("""
                CREATE TABLE produto(
                nome  VARCHAR2(255) NOT NULL ,
                descricao VARCHAR2(255),
                codigo VARCHAR2(30) NOT NULL PRIMARY KEY,
                custo INTEGER NOT NULL,
                custoFixo INTEGER NOT NULL,
                comissao INTEGER NOT NULL,
                imposto INTEGER NOT NULL,
                rentabilidade INTEGER NOT NULL 
                )"""
)

connection.commit()


DatabaseError: ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/

In [10]:
cursor = connection.cursor()

cursor.execute('''
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade)
                VALUES ('Caneta', 'Caneta Profissional', '1', 36, 15, 5, 12, 20);
                ''')
connection.commit()


DatabaseError: ORA-00933: SQL command not properly ended
Help: https://docs.oracle.com/error-help/db/ora-00933/